# How to run TARP

TARP is an alternative calibration check proposed recently in https://arxiv.org/abs/2302.03026. 

In contrast to SBC (Talts et al.) and expected coverage based highest posterior density regions
(Deistler et al.,), TARP provides a necessary *and sufficient* condition for posterior
accuracy, i.e., it can also detect inaccurate posterior estimators. 

Note, however, that this property depends on the choice of reference point distribution:
to obtain the full diagnostic power of TARP, one would need to sample reference points
from a distribution that depends on $x$. Thus, in general, we recommend using and
interpreting TARP like SBC and complementing coverage checks with posterior predictive
checks.

You can run TARP in the `sbi` toolbox as follows:

```python
from sbi.diagnostics import run_tarp
from sbi.analysis.plot import plot_tarp

num_tarp_samples = 200  # choose a number of sbc runs, should be ~100s
# generate ground truth parameters and corresponding simulated observations for SBC.
thetas = prior.sample((num_tarp_samples,))
xs = simulator(thetas)

# the tarp method returns the ECP values for a given set of alpha coverage levels.
ecp, alpha = run_tarp(
    thetas,
    xs,
    posterior,
    references=None,  # will be calculated automatically.
    num_posterior_samples=1000,
)
# Similar to SBC, we can check then check whether the distribution of ecp is close to
# that of alpha.
atc, ks_pval = check_tarp(ecp, alpha)
print(atc, "Should be close to 0")
print(ks_pval, "Should be larger than 0.05")

plot_tarp(ecp, alpha)
```

This generates a plot like the following:

<img src="tarp_plot.png" width="400">

If the blue curve is above the diagonal, then the posterior estimate is under-confident. If it is under the diagonal, then the posterior estimate is over confident.

## Explanation

Given a test set $(\theta^*, x^*)$ and a set of reference points $\theta_r$, TARP calculates 
statistics for posterior calibration by 
- drawing posterior samples $\theta$ given each $x_*$
- calculating the distance $r$ between $\theta_*$ and $\theta_r$
- counting for how many of the posterior samples their distance to $\theta_r$ is smaller than $r$

See https://arxiv.org/abs/2302.03026, Figure 2, for an illustration.

For each given coverage level $\alpha$, one can then calculate the corresponding average counts
and check, whether they correspond to the given $\alpha$. 